In [1]:
%load_ext jupyter_black

In [2]:
import os
from pathlib import Path
from typing import Iterator
from datetime import datetime
from shutil import copyfileobj

from requests import Session, HTTPError
import pandas as pd

In [3]:
def google_api_hrrr_grib2_data(start: datetime, end: datetime) -> Iterator[tuple[str, str]]:
    """
    generate a date range on an hourly interval
    """

    base_url = "https://storage.googleapis.com/high-resolution-rapid-refresh/"
    date_range = pd.date_range(start, end, freq="h")

    yield from zip(
        date_range.strftime("%Y-%m-%dT%H:%M:%SZ"),
        base_url + date_range.strftime("hrrr.%Y%m%d/conus/hrrr.t%Hz.wrfnatf00.grib2"),
    )

In [6]:
def download_and_gribs(start: datetime, end: datetime, path: Path) -> list[str]:
    """
    iterate over validtimes and urls and save files to a Path directory
    returns a list of urls that failed to download
    """
    failed_files = []
    # request context manager
    with Session() as session:
        # iteratate over the generator function
        for validtime, url in google_api_hrrr_grib2_data(start=start, end=end):
            # add the filename to the path object
            save_to = path / f"{validtime}.grib2"
            try:
                # make a http get request to the url
                res = session.get(url, stream=True)
                # on non 200 status code raise HTTPError
                res.raise_for_status()
                # save the file to the directory
                with save_to.open("wb") as fileout:
                    copyfileobj(res.raw, fileout)
                print("grib2 file saved at ", save_to)

            except HTTPError:
                # on a failed http reqest append the url to the failed_files list
                failed_files.append(url)

    return failed_files


media_ext = Path("/workspaces/griblib/data/hrrr/")

download_and_gribs(start=datetime(2022, 6, 18), end=datetime(2022, 6, 19), path=media_ext)

grib2 file saved at  /workspaces/griblib/data/hrrr/2022-06-18T00:00:00Z.grib2
grib2 file saved at  /workspaces/griblib/data/hrrr/2022-06-18T01:00:00Z.grib2
grib2 file saved at  /workspaces/griblib/data/hrrr/2022-06-18T02:00:00Z.grib2
grib2 file saved at  /workspaces/griblib/data/hrrr/2022-06-18T03:00:00Z.grib2
grib2 file saved at  /workspaces/griblib/data/hrrr/2022-06-18T04:00:00Z.grib2
grib2 file saved at  /workspaces/griblib/data/hrrr/2022-06-18T05:00:00Z.grib2
grib2 file saved at  /workspaces/griblib/data/hrrr/2022-06-18T06:00:00Z.grib2
grib2 file saved at  /workspaces/griblib/data/hrrr/2022-06-18T07:00:00Z.grib2
grib2 file saved at  /workspaces/griblib/data/hrrr/2022-06-18T08:00:00Z.grib2
grib2 file saved at  /workspaces/griblib/data/hrrr/2022-06-18T09:00:00Z.grib2
grib2 file saved at  /workspaces/griblib/data/hrrr/2022-06-18T10:00:00Z.grib2
grib2 file saved at  /workspaces/griblib/data/hrrr/2022-06-18T11:00:00Z.grib2
grib2 file saved at  /workspaces/griblib/data/hrrr/2022-06-18T12

[]